# Un ejemplo un poco más complicado

Vamos a hacer un caso muy simplificado de una cuenta de un banco.

## Índice
- Protocolo de mensajería
  + De entrada
  + De salida
  + Eventos
- Cuentas y transferencias
  + Actor cuenta
  + Interfaz cuenta
  + Definición de una transferencia
- Implementaciones
  + Logica de negocio: actualización del balance
  + Publicación de eventos
 

###  Se importan la librerias de akka 

In [1]:
import $ivy.`com.typesafe.akka::akka-actor:2.5.14`

import $ivy.$                                     

### Implicitos necesarios

In [2]:
import scala.concurrent.ExecutionContext 
import java.util.concurrent.Executors
import akka.util.Timeout
import scala.concurrent.duration._

implicit val ec  = ExecutionContext.fromExecutorService( Executors.newFixedThreadPool( 20 ) )
implicit val timeout = Timeout( 5 seconds )   

import scala.concurrent.ExecutionContext 

import java.util.concurrent.Executors

import akka.util.Timeout

import scala.concurrent.duration._


ec: concurrent.ExecutionContextExecutorService = scala.concurrent.impl.ExecutionContextImpl$$anon$1@2ed0f8c1
timeout: akka.util.Timeout = Timeout(5 seconds)

---

---
## Se define el protocolo de mensajería

### Protocolo de entrada

In [3]:
sealed trait AccountIn

// Commands

sealed trait AccountCommand extends AccountIn { 
    val amount : Int
}

final case class Withdrawal(amount : Int) extends AccountCommand 
final case class Income(amount : Int) extends AccountCommand 


// Queries

sealed trait AccountQuery extends AccountIn

final case object GetBalance extends AccountQuery

defined trait AccountIn
defined trait AccountCommand
defined class Withdrawal
defined class Income
defined trait AccountQuery
defined object GetBalance

### Protocolo de salida

In [4]:
sealed trait AccountOut

final case class CurrentBalance( balance: Int ) extends AccountOut
final case class DeltaBalance( delta : Int ) extends AccountOut

defined trait AccountOut
defined class CurrentBalance
defined class DeltaBalance

### Eventos

In [5]:
sealed trait AccountEvent {
    
    val idAccount : String
    val amount: Int
}

case class WithdrawalCreated( val idAccount: String, val amount : Int ) extends AccountEvent
case class IncomeCreated( val idAccount: String, val amount : Int ) extends AccountEvent

defined trait AccountEvent
defined class WithdrawalCreated
defined class IncomeCreated

---

---
## Cuentas y transferencias
### Actor 'Cuenta'

In [6]:
import akka.actor._
import scala.collection.mutable.Queue
import scala.util._

class ActorAccount( private val updateBalance : (Int, Int) => Try[Int], 
                       private val queueCQRS: Queue[AccountEvent] ) extends Actor {
    
    
    val id = self.path.name
    
    var balance : Int = 0
    
    override def receive = {
        
        case command : AccountCommand => manageCommads( command )
        case querry  : AccountQuery   => manageQueries( querry )
        case other                    => unhandled( other )
        
    }
    
    private def manageCommads( command: AccountCommand ) : Unit = {
        
        command match {
            case Withdrawal( amount ) => execUpdateBalance( -1 * amount, command)
            case Income( amount )     => execUpdateBalance( amount, command )
            
        }           
        
    }
    
    private def execUpdateBalance( amount : Int, command : AccountCommand) = {        
        updateBalance( amount, balance ) match {            
            case Success( newBalance ) => {
                sender() ! DeltaBalance( newBalance - balance ) 
                sendEvent( command )
                balance = newBalance
            }
            case Failure(  error ) => sender() ! Status.Failure( error )         
        }
    }
    
    private def sendEvent( command: AccountCommand ) {
        
        val event : AccountEvent = command match {
            case Withdrawal( amount ) => WithdrawalCreated( id, amount ) 
            case Income( amount )     => IncomeCreated( id, amount ) 
        }
        
        queueCQRS.enqueue( event )
        
    }
    
    private def manageQueries( queries : AccountQuery ) : Unit = queries match {
        case GetBalance => sender() !  CurrentBalance( balance )
    }
    
    
}

import akka.actor._

import scala.collection.mutable.Queue

import scala.util._


defined class ActorAccount

### Interfaz 'Cuenta'

In [7]:
import scala.concurrent.Future

trait Account {
   def makeWithdrawal( amount : Int ) : Future[ Int ] 
   def makeIncome( amount : Int ) : Future[Int] 
   def getBalance: Future[Int]
}

object Account {
    
    import akka.pattern._
    import akka.actor._
    import akka.util.Timeout
    
    def apply( account :ActorRef )
                    ( implicit ec : ExecutionContext, timeout : Timeout ) = new Account {
      
     def makeWithdrawal( amount : Int ) : Future[ Int] = {
        ( account ? Withdrawal( amount ) ).mapTo[DeltaBalance].map( _.delta )            
     }

     def makeIncome( amount : Int ) : Future[Int] = {
            ( account ? Income( amount ) ).mapTo[DeltaBalance].map( _.delta )             
     }
        
      def getBalance : Future[Int] = {
         ( account ? GetBalance ). mapTo[ CurrentBalance ].map( _.balance ) 
      }
        
    }
}



import scala.concurrent.Future


defined trait Account
defined object Account

### Defincición de una transferencia

Se simula una operación/compensacion siguiendo el patrón sagas

In [8]:
object Transfer {
    
    import scala.concurrent._
    
    def transfer( from : Account, to: Account )( amount : Int )( implicit ec : ExecutionContext) = {
        
        from.makeWithdrawal( amount ).flatMap {
             _ => to.makeIncome( amount )
                    .map( _ => true )
                    .recoverWith{ 
                        case _ => from.makeIncome( amount )
                                      .map( _=> false ) 
                    }
        }
        
   }
    
}



defined object Transfer

---

----
## Implementaciones

### Lógica de negocio
Se define una lógica de negocio simple. En este caso no se admiten descubiertos, pero por ejemplo se pueden implementar diferentes lógicas como un porcentaje de descubierto dependiendo del balance. 
> El objetivo final es que la lógica puede estar separada del actor y puede ser validada y probada aparte

In [9]:
import scala.util._

implicit val updateBalance : (Int,Int) => Try[Int] = ( amount, balance ) => {
   
    val newBalance = amount + balance
    
    if( newBalance >= 0 ) {
    
        Success( newBalance )
        
    } else {
        
        Failure( new IllegalStateException( s"It should not be in red( ${newBalance} )" ) )
    }
    
}

import scala.util._


updateBalance: (Int, Int) => Try[Int] = $sess.cmd8Wrapper$Helper$$Lambda$3398/364795502@d0a781

### Indirección de publicación de eventos
Se define una cola que será la indirección de publicación de eventos.
En este caso para esta prueba será una cola mitable de Scala.   
> En un sistema real puede ser un akka stream con su fuente '_materializada_' en una cola

In [10]:
import scala.collection.mutable.Queue

val queueCQRS = Queue[AccountEvent]()

import scala.collection.mutable.Queue


queueCQRS: Queue[AccountEvent] = Queue()

---

---
## Probandolo todo

### _Testing: Utilidades_

In [11]:
object TestUtil {
    
    import scala.concurrent._, duration._
    import akka.pattern._
    import akka.util.Timeout


    val tm = 5 seconds
    implicit val timeout = Timeout( tm )

    def result[T]( future : => Future[T] ) = Try {
        Await.result( future, tm )
    }
    
}

defined object TestUtil

---
### Iniciando el entorno

#### Se crea el sistema de actores

In [12]:
val system = akka.actor.ActorSystem.create( "test-1" )

system: ActorSystem = akka://test-1

#### Se crean dos actores cuenta

In [13]:
import akka.actor.Props

val accountOneActor = system.actorOf (
        Props( new ActorAccount(  updateBalance, queueCQRS )),  
       "accountOne"  
)

val accountTwoActor = system.actorOf (
        Props( new ActorAccount( updateBalance, queueCQRS ) ),
        "accountTwo"
)

import akka.actor.Props


accountOneActor: ActorRef = Actor[akka://test-1/user/accountOne#289101094]
accountTwoActor: ActorRef = Actor[akka://test-1/user/accountTwo#-722463502]

#### Se crean dos 'entidades' cuenta

In [14]:
val accountOne = Account( accountOneActor )
val accountTwo = Account( accountTwoActor )

accountOne: AnyRef with Account = $sess.cmd6Wrapper$Helper$Account$$anon$1@29a51948
accountTwo: AnyRef with Account = $sess.cmd6Wrapper$Helper$Account$$anon$1@4f90a960

#### Se hace un ingreso incial a las dos cuentas

Después se comprueba el balance y se obtiene el total del dinero (la suma de los dos balances)

> Aquí se hace `Await` sólo por motivos de testing

In [15]:

TestUtil.result{
    for {
        _  <- accountOne.makeIncome( 1000 )
        _  <- accountTwo.makeIncome( 1000 ) 
        b1 <- accountOne.getBalance  
        b2 <- accountTwo.getBalance 
    } yield {( b1, b2, b1 + b2 )}
}

res14: Try[(Int, Int, Int)] = Success((1000, 1000, 2000))

#### Se comprueban los eventos

In [16]:
queueCQRS.toList ; queueCQRS.clear

res15_0: List[AccountEvent] = List(IncomeCreated(accountOne,1000), IncomeCreated(accountTwo,1000))

---
### _Probando, probando_

#### Funciones de utilidades
Para poder testear transferencias de una cuenta a otra de una manera más cómoda

In [17]:
val transfersOneToTwo =  Transfer.transfer( accountOne, accountTwo)( _ )
val transfersTwoToOne =  Transfer.transfer( accountTwo, accountOne)( _ )

transfersOneToTwo: Int => Future[Boolean] = $sess.cmd16Wrapper$Helper$$Lambda$3646/472174427@ad26fb4
transfersTwoToOne: Int => Future[Boolean] = $sess.cmd16Wrapper$Helper$$Lambda$3647/549467421@5898b0a2

#### Primera prueba

In [18]:
TestUtil.result{
       for {
           a <- transfersOneToTwo( 500 ) 
           b <- transfersTwoToOne( 500 ) 
       } yield( a && b )
}

res17: Try[Boolean] = Success(true)

#### Se vuelen a compruebar los balances

> `Await` sólo por motivos de testing

In [19]:
TestUtil.result {
    for {
        b1 <- accountOne.getBalance
        b2 <- accountTwo.getBalance  
    } yield{  (b1, b2, b1 +b2) }    
}

res18: Try[(Int, Int, Int)] = Success((1000, 1000, 2000))

#### Se vuelven a comprobar los eventos

In [20]:
queueCQRS.toList ; queueCQRS.clear

res19_0: List[AccountEvent] = List(
  WithdrawalCreated(accountOne,500),
  IncomeCreated(accountTwo,500),
  WithdrawalCreated(accountTwo,500),
  IncomeCreated(accountOne,500)
)

---
### Bonus track
Test de concurrencia, lanzando transferencias en paralelo desde las mismas cuentas con cantidades aleatorias.

El resultado debera conservar la suma de los balances y ninguna de las dos cuentas puede tener valores negativos, ni valores mayores que la suma de los dos balances inciales

In [21]:
object TestConcurrent {

    import scala.util.Random

    def randomTransfer( implicit ec : ExecutionContext ) = Future {
        val amount = Random.nextInt( 998 ) + 1
        if ( Random.nextInt(2) == 0 ) {
            transfersOneToTwo( amount )
        } else {
            transfersTwoToOne( amount )
        }
    }.flatten.map{
       _ => 1  
    } .recover{
        case _=> 0
    }
  
    def testConncurrent( cont: Int, 
                         from : List[Future[Int]] = List())( implicit ec : ExecutionContext ): Future[Int] ={

        if ( cont == 0 ) {
            Future.sequence( from ) .map {
                _.fold(0) { 
                        ( a,b ) => a + b 
                } 
            }
        } else  {
           val newRandomTransfer = randomTransfer    
           testConncurrent( cont -1, newRandomTransfer +: from )
        }
    }

}

defined object TestConcurrent

In [22]:
TestUtil.result {
    for {
        t <- TestConcurrent.testConncurrent( 500 ) 
        b1 <- accountOne.getBalance
        b2 <- accountTwo.getBalance  
    } yield{  (t, (b1, b2, b1 +b2) ) }    
}

res21: Try[(Int, (Int, Int, Int))] = Success((19, (1487, 513, 2000)))

#### Se vuelven a comprobar los eventos

In [23]:
queueCQRS.toList ; queueCQRS.clear

res22_0: List[AccountEvent] = List(
  WithdrawalCreated(accountOne,956),
  WithdrawalCreated(accountTwo,609),
  WithdrawalCreated(accountTwo,257),
  IncomeCreated(accountOne,609),
  IncomeCreated(accountOne,257),
  WithdrawalCreated(accountOne,470),
  WithdrawalCreated(accountOne,232),
  WithdrawalCreated(accountOne,62),
  WithdrawalCreated(accountTwo,6),
  WithdrawalCreated(accountTwo,92),
  IncomeCreated(accountTwo,956),
...